In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
ds = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
Model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(batch):
    source = batch['dialogue']
    target = batch['summary']

    # Tokenize the source (input) dialogue
    source_ids = tokenizer(source, truncation=True, padding='max_length', max_length=128)

    # Tokenize the target (output) summary
    target_ids = tokenizer(target, truncation=True, padding='max_length', max_length=128)

    # Extract input IDs to use as labels
    labels = target_ids['input_ids']

    # Set padding tokens in labels to -100 so they are ignored by the loss function
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels]

    return {
        "input_ids": source_ids["input_ids"],
        "attention_mask": source_ids["attention_mask"],
        "labels": labels
    }

In [ ]:
df_source=ds.map(preprocess_function,batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
# training_arguments
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    remove_unused_columns=True
)

In [ ]:
trainer= Trainer(
    model=Model,
    args=training_args,
    train_dataset=df_source['train'],
    eval_dataset=df_source['test']
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jabinjames2002 (jabinjames2002-mar-baselios-college-of-engineering-and-t) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,1.591500
1000,1.488400
1500,1.433500
2000,1.084000
2500,1.019900
3000,1.001400


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3116, training_loss=1.261064815888509, metrics={'train_runtime': 3761.76, 'train_samples_per_second': 6.625, 'train_steps_per_second': 0.828, 'total_flos': 6750530835578880.0, 'train_loss': 1.261064815888509, 'epoch': 2.0})

In [ ]:
eval_results=trainer.evaluate()

In [ ]:
eval_results

{'eval_loss': 1.664028286933899,
 'eval_runtime': 53.0463,
 'eval_samples_per_second': 28.277,
 'eval_steps_per_second': 3.544,
 'epoch': 2.0}

In [ ]:
Model.save_pretrained('/content/model_directory')
tokenizer.save_pretrained('/content/model_directory')

('/content/model_directory/tokenizer_config.json',
 '/content/model_directory/special_tokens_map.json',
 '/content/model_directory/vocab.json',
 '/content/model_directory/merges.txt',
 '/content/model_directory/added_tokens.json',
 '/content/model_directory/tokenizer.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/model_directory")
model=AutoModelForSeq2SeqLM.from_pretrained("/content/model_directory")

/usr/local/lib/python3.12/dist-packages/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [ ]:
def summazize(blog_post):
  inputs=tokenizer(blog_post,max_length=1024,truncation=True,return_tensors='pt')

  # Move the input IDs to the same device as the model
  input_ids_on_device = inputs['input_ids'].to(Model.device)

  summary_ids=Model.generate(input_ids_on_device,max_length=150,min_length=40,length_penalty=2.0,num_beams=4,early_stopping=True)
  print(f"Generated Summary IDs: {summary_ids}")
  summary=tokenizer.decode(summary_ids[0],skip_special_tokens=True)

  return summary

In [ ]:
blog_posts="""
Retrieval-Augmented Generation (RAG) is an advanced AI architecture designed to improve the accuracy, reliability, and factual grounding of large language models by combining information retrieval with text generation. Instead of relying solely on a model’s pre-trained knowledge, RAG retrieves relevant documents from an external knowledge base at query time and uses them as contextual input for response generation. This approach significantly reduces hallucinations, enables access to up-to-date or domain-specific information, and removes the need for frequent model retraining. A typical RAG pipeline involves document ingestion, text chunking, embedding generation, vector storage, semantic retrieval, and final response generation using a language model. By grounding outputs in retrieved evidence, RAG is particularly effective for applications such as question answering, enterprise search, chatbots, and knowledge assistants. Overall, RAG provides a scalable and cost-effective way to build intelligent systems that require both strong reasoning capabilities and trustworthy, context-aware responses
"""

In [ ]:
summary=summazize(blog_posts)

Generated Summary IDs: tensor([[    2,     0,     0,     0, 10431, 40456,    16,    41,  3319,  4687,
          9437,  1887,     7,  1477,     5,  8611,     6, 13677,     6,     8,
         21833, 36788,     9,   739,  2777,  3092,    30, 15224,   335, 43372,
            19,  2788,  2706,     4,   849, 40456,  1639,    10, 31909,     8,
           701,    12, 20278,   169,     7,  1119, 12038,  1743,    14,  2703,
           258,   670, 20511,  5587,     8, 32101,     6,  5377,    12, 24590,
          8823,     4,     2]], device='cuda:0')


In [ ]:
print(summary)

#RAG is an advanced AI architecture designed to improve the accuracy, reliability, and factual grounding of large language models by combining information retrieval with text generation. #RAG provides a scalable and cost-effective way to build intelligent systems that require both strong reasoning capabilities and trustworthy, context-aware responses.


# New Section